In [133]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [156]:
urls = [
    'https://www.hepsiburada.com/rampage-sn-r9-oyuncu-siyah-kirmizi-mikrofonlu-kulaklik-p-BDUCZLKPC-28753-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/1more-xiaomi-1more-piston-fit-kulak-ici-kulaklik-e1009-p-HBV00000NSSKY-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/evocase-evo300-15-6-notebook-cantasi-evo301-bilek-destekli-mousepad-kablosuz-mouse-set-p-HBV00000NFDTT-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/general-mobile-gm5-plus-general-mobile-garantili-p-TELCEPGM5PT-RG-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/samsung-e1190-p-TELCEPSAME1190-K-yorumlari?sayfa={}&sadeceonayli=evet'
   ]

In [ ]:
"""
'https://www.hepsiburada.com/dark-rangemax-wrt360-300mbit-2x3dbi-dahili-antenli-802-11n-wifi-mini-repeater-dk-nt-wrt360-p-HBV00000850Z8-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/lexmark-mb2236adw-fotokopi-tarama-fax-dubleks-wifi-monochrome-cok-fonksiyonlu-lazer-yazici-p-HBV00000RQDAW-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/jbl-c100siublk-mikrofonlu-kulakici-kulaklik-ct-ie-siyah-p-HBV00000201EP-yorumlari?sayfa={}&sadeceonayli=evet',
'https://www.hepsiburada.com/everest-w777-usb-2-4ghz-optik-wireless-mouse-kirmizi-p-HBV00000A8KM9-yorumlari?sayfa={}&sadeceonayli=evet',
   'https://www.hepsiburada.com/everest-km-191-entry-usb-aydinlatmali-oyuncu-klavye-mouse-set-p-HBV00000SSKHT-yorumlari?sayfa={}&sadeceonayli=evet',
'https://www.hepsiburada.com/xiaomi-redmi-airdots-basic-2-tws-bluetooth-5-0-kulaklik-p-HBV00000WGFD4-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/ixtech-thinbook-intel-atom-z3735f-2gb-32gb-emmc-windows-10-home-11-6-fhd-tasinabilir-bilgisayar-p-HBV00000PO6T7-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/mmctech-m4-akilli-bileklik-p-HBV00000MLW9D-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/samsung-ta20-adaptec-fast-charge-hizli-seyahat-sarji-micro-cikisli-beyaz-samsung-turkiye-garantilidir-p-HBV000003KBMX-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/arcelik-ac-6030-havadar-uzaktan-kumandali-iyonizer-hava-sogutucu-p-hbv00000lkpnz-yorumlari?sayfa={}&sadeceonayli=evet',
     'https://www.hepsiburada.com/kingston-datatraveler-106-32gb-usb-3-0-bellek-dt106-32gb-p-HBV00000EF1QT-yorumlari?sayfa={}&sadeceonayli=evet',
    'https://www.hepsiburada.com/awox-u5100-50-127-ekran-uydu-alicili-4k-smart-d-led-tv-p-HBV00000RTQ7W-yorumlari?sayfa={}&sadeceonayli=evet'
"""

In [19]:
def filter_content_holder(holder):
    description = holder.find('span', {"itemprop":"description"})
    return True if description else False

In [42]:
def get_comment_stars(comment):
    return len(comment.find_all('path', {"fill": '#f28b00'}))

In [55]:
def filter_comments_by_stars(comment):
    stars = get_comment_stars(comment)
    return True if stars == 5 or stars == 1 else False

In [56]:
def get_comment_sentiment(comment):
    stars = get_comment_stars(comment)
    return 'Olumlu' if stars == 5 else 'Olumsuz'

In [142]:
def deEmojify(inputString):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',inputString)

In [157]:
for url in urls:
    response = requests.get(url.format(1), headers={'User-Agent': 'Custom'})
    soup = BeautifulSoup(response.text, 'html.parser')
    last_page = int(soup.find('div', {"class":"paginationBarHolder"}).findAll('li')[-1].text)
    comments = []
    sentiments = []
    for i in range(1, last_page + 1):
        response = requests.get(url.format(i), headers={'User-Agent': 'Custom'})
        soup = BeautifulSoup(response.text, 'html.parser')
        content_holder = soup.find('div', {"class": "paginationContentHolder"})
        comments_holder  = list(filter(filter_content_holder, content_holder.find_all('div', recursive=False)))
        comments_holder = list(filter(filter_comments_by_stars, comments_holder))
        for comment in comments_holder:
            comments.append(deEmojify(comment.find('span', {"itemprop":"description"}).text))
            sentiments.append(get_comment_sentiment(comment))
    comments_list = [[comment, sentiment] for comment, sentiment in zip(comments, sentiments)]
    comments_array = np.array(comments_list)
    max_comments = len(comments_array[comments_array[:,1] == 'Olumlu']) if len(comments_array[comments_array[:,1] == 'Olumlu']) < len(comments_array[comments_array[:,1] == 'Olumsuz']) else len(comments_array[comments_array[:,1] == 'Olumsuz'])
    np.random.shuffle(comments_array)
    minimized_array = np.concatenate((comments_array[comments_array[:,1] == 'Olumlu'][:max_comments], comments_array[comments_array[:,1] == 'Olumsuz'][:max_comments]))
    np.random.shuffle(minimized_array)
    comments_df = pd.DataFrame(minimized_array, columns=['Görüş', 'Durum'])
    if os.path.exists('yorumlar.csv'):
        comments_df.to_csv('yorumlar.csv', mode='a', header=False, index=False)
    else:
        comments_df.to_csv('yorumlar.csv', index=False)
    print('Done:',url)

Done: https://www.hepsiburada.com/rampage-sn-r9-oyuncu-siyah-kirmizi-mikrofonlu-kulaklik-p-BDUCZLKPC-28753-yorumlari?sayfa={}&sadeceonayli=evet
Done: https://www.hepsiburada.com/1more-xiaomi-1more-piston-fit-kulak-ici-kulaklik-e1009-p-HBV00000NSSKY-yorumlari?sayfa={}&sadeceonayli=evet
Done: https://www.hepsiburada.com/evocase-evo300-15-6-notebook-cantasi-evo301-bilek-destekli-mousepad-kablosuz-mouse-set-p-HBV00000NFDTT-yorumlari?sayfa={}&sadeceonayli=evet
Done: https://www.hepsiburada.com/general-mobile-gm5-plus-general-mobile-garantili-p-TELCEPGM5PT-RG-yorumlari?sayfa={}&sadeceonayli=evet
Done: https://www.hepsiburada.com/samsung-e1190-p-TELCEPSAME1190-K-yorumlari?sayfa={}&sadeceonayli=evet
